In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade pymilvus "pymilvus[model]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.8 MB/s eta 0:00:00


In [3]:
import json
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    AnnSearchRequest,
    WeightedRanker,
    MilvusClient
)
from sentence_transformers import CrossEncoder

In [ ]:
with open('/content/drive/MyDrive/data/data.json', 'r', encoding='utf-8') as file:
    stored_chunks = json.load(file)

In [22]:
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cuda")
dense_dim = ef.dim["dense"]
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device='cuda')

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [5]:
connections.connect(uri="", token="")
col_name = 'ing_agentic_ai'

In [6]:
if not utility.has_collection(col_name):
    fields = [
        FieldSchema(name="chunk_id", dtype=DataType.VARCHAR, is_primary=True, max_length=128),
        FieldSchema(name="page_content", dtype=DataType.VARCHAR, max_length=10240),
        FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=64),
        FieldSchema(name="page", dtype=DataType.INT32),
        FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=64),
        FieldSchema(name="char_start", dtype=DataType.INT32),
        FieldSchema(name="char_end", dtype=DataType.INT32),
        FieldSchema(name="parent_heading", dtype=DataType.VARCHAR, max_length=128, nullable=True),
        FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
        FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
    ]
    schema = CollectionSchema(fields)
    col = Collection(col_name, schema, consistency_level="Bounded")
    sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
    col.create_index("sparse_vector", sparse_index)
    dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
    col.create_index("dense_vector", dense_index)
else:
    col = Collection(col_name)

col.load()

In [ ]:
# batch_size = 50
# length = len(stored_chunks)
# for i in range(0, length, batch_size):
#     batch = stored_chunks[i:i+batch_size]
#     content_list = [chunk['page_content'] for chunk in batch]
#     docs_embeddings = ef(content_list)
#     batched_entities = [
#             [chunk['chunk_id'] for chunk in batch],
#             content_list,
#             [chunk['source'] for chunk in batch],
#             [chunk['page'] for chunk in batch],
#             [chunk['type'] for chunk in batch],
#             [chunk['char_start'] for chunk in batch],
#             [chunk['char_end'] for chunk in batch],
#             [chunk['parent_heading'] for chunk in batch],
#             docs_embeddings["sparse"],
#             docs_embeddings["dense"],
#     ]
#     col.insert(batched_entities)
# print(f"Number of entities inserted:{col.num_entities} / {length}")

Inference Embeddings: 100%|██████████| 4/4 [00:04<00:00,  1.10s/it]


Number of entities inserted:0 / 213


In [26]:
def query_embedding(query):
    query_embeddings = ef([query])
    return query_embeddings["sparse"][[0]], query_embeddings["dense"][0]

def output_fields():
    return ['chunk_id', 'page_content', 'source', 'page', 'type', 'char_start', 'char_end', 'parent_heading']

def rerank(query, results):
    pairs = [[query, result['page_content']] for result in results]
    scores = reranker.predict(pairs)
    for r, s in zip(results, scores):
        r['score'] = s
    reranked_results = sorted(results, key=lambda x: r['score'], reverse=True)
    return reranked_results

def top_k_sparse_search(query, top_k=5):
    query_sparse_embedding, query_dense_embedding = query_embedding(query)
    result = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=2 * top_k,
        output_fields=output_fields(),
        param={"metric_type": "IP", "params": {}},
    )[0]
    chunks = [record['entity'] for record in result]
    return rerank(query, chunks)

In [28]:
res = top_k_sparse_search('sustainability')
res

[{'char_start': 0,
  'char_end': 1233,
  'parent_heading': '(Text with EEA relevance)',
  'page_content': 'to meet the information needs of financial market participants subject to the requirements laid down in the delegated acts adopted pursuant to Article 4(6) and (7) of Regulation (EU) 2019/2088, the Commission should adopt a first set of sustainability reporting standards by means of delegated acts by 30 June 2023. That set of sustainability reporting standards should specify the information that undertakings should disclose with regard to all reporting areas and sustainability matters, and that financial market participants need to comply with the disclosure obligations laid down in Regulation (EU) 2019/2088. The Commission should adopt a second set of sustainability reporting standards by means of delegated acts by 30 June 2024, specifying complementary information that undertakings should disclose about sustainability matters and reporting areas, where necessary, and information